In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Define the path to save the fine-tuned model
load_model = os.path.join("..", "..", "models", "fine_tuned_model")  # Points to InsureAI/models/fine_tuned_model

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(load_model)
tokenizer = AutoTokenizer.from_pretrained(load_model)

# Set pad_token_id to eos_token_id for open-end generation
model.config.pad_token_id = model.config.eos_token_id  # Ensures padding uses the eos_token_id

# Generate a response
prompt = "Tell me about the company InsureAI."
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=512)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tell me about the company InsureAI.

InsureAI is a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a company that is based in India. We are a compan